In [1]:
import pandas as pd
import pm4py

In [2]:
application_log = pm4py.read_xes('BPI Challenge 2017.xes')
offer_log = pm4py.read_xes('BPI Challenge 2017 - Offer log.xes')

df_application = pm4py.convert_to_dataframe(application_log)
df_offer = pm4py.convert_to_dataframe(offer_log)

df_application.to_csv('app_logs.csv')
df_offer.to_csv('offer_logs.csv')

parsing log, completed traces :: 100%|██████████| 42995/42995 [00:10<00:00, 4021.40it/s]


In [2]:
df_application = pd.read_csv('./../../Assignment_2/Data/app_logs.csv')
df_offer = pd.read_csv('./../../Assignment_2/Data/offer_logs.csv')

In [3]:
declined_ids = list(df_application.loc[df_application['concept:name'] == 'A_Denied']['case:concept:name'].unique())
accepted_ids = list(df_application.loc[df_application['concept:name'] == 'A_Pending']['case:concept:name'].unique())
cancelled_ids = list(df_application.loc[df_application['concept:name'] == 'A_Cancelled']['case:concept:name'].unique())

In [5]:
# 31,509 loan applications in total
# 98 noise 
len(declined_ids)+ len(accepted_ids)+len(cancelled_ids)

31411

# Part 2 building dataset

In [ ]:
def aggregate_df(df):
    """ 
    Aggregate the df of current events in the case

    Output: 
        result -> could be a pandas series
    """

    # record the timestamp of the last activity
    result = df.iloc[0]

    return result


In [ ]:
def add_to_aggregate(result, df_row):
    """ 
    When a new event happens, add the event info to the current aggregated result.

    Input: 
         result: the current aggregated result
         df_row: pandas df row representing the new event
    Output:
        result: the new aggregated result
    """

    # record the timestamp of the last activity
    result = df_row.iloc[0]

    return result

In [34]:
def create_prefix_part2(df_application, app_ids, end_event, start_event='A_Accepted'):
    
    app_id_list = list(df_application['case:concept:name'].unique())

    # TODO:
    # create a return df
    return_df = pd.DataFrame()

    # extracting prefix for each application
    for app_id in app_id_list:
        
        events_app = df_application.loc[df_application['case:concept:name'] == app_id]
        events_app.reset_index(drop=True, inplace=True)

        cur_id = starting_row_id = events_app.loc[events_app['concept:name'] == 'A_Accepted'].index[0]
        pre_events = events_app.iloc[:starting_row_id]
        # TODO: 
        # aggregate events_app from row 0 to starting_row_id
        result = aggregate_df(pre_events)
        
        ending_row_id = events_app.loc[events_app['concept:name'] == end_event].index[0]
        cur_id += 1
        
        while cur_id < ending_row_id:
            new_row = events_app.iloc[cur_id]
            # TODO: 
            # add new event row info to the aggregated result
            result = add_to_aggregate(pre_events)

            # Update the return_df -> add new row
            # target y: end_event

            cur_id += 1

        return return_df


# Part 3 building dataset


In [35]:
# ## similar to above, change the target y to the duration of the case

# Part 4 building dataset

In [25]:
new_df = df_application.loc[df_application['case:concept:name'] == 'Application_428409768']

In [ ]:
new_df.reset_index(drop=True, inplace=True)

In [ ]:
new_df

In [ ]:
def create_prefix(df_application, current_event, target_event):
    """
    Encode original applications 
    """
    
    app_id_list = list(df_application['case:concept:name'].unique())

    # TODO: get all unique activities before target_event
    # TODO: get all unique resources before target_event
    # use abrove info to aggregate below

    for app_id in app_id_list:

        all_events = df_application.loc[df_application['case:concept:name'] == app_id]

        # aggregate rows of events and append the timestamp of the target_event

        # Encode each case attribute as a feature (or one-hot encode categorical case attributes)

        # For each numerical event attribute, apply an aggregation function (e.g. average) over the sequence of values taken by this attribute in the prefix
        # sum up time as one feature?

        # For each categorical event attribute, encode each possible value of that attribute as a numerical feature. 
        
        
        